In [29]:
import numpy as np
from math import e
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [61]:
class Calculate_xjLs:
    def calculate_xjLs(self,x):
        xjLs = [x]
        sjLs = []
        for layer in range(self.N_layers-1):
            x_l = [1] if layer != self.N_layers-2 else [] #self.N_layers-2 bc layer 2nd gives layer 3 and layer 2nd is actually 1
            s_l = []
            weights_for_layer = self.weights[layer]
            for w_jl in weights_for_layer: #picking specific weights of one output (j)
                s_jl = np.matmul(x,w_jl.T)
                self.count += (len(x))
                x_jl = self.activation_func(np.matmul(x,w_jl.T))
                x_l.append(x_jl)
                s_l.append(s_jl)
            xjLs.append(x_l)
            sjLs.append(s_l)
            x = x_l
        return xjLs, sjLs

class CalculateDeltas:
    def calculate_deltaL(self,x,y):
        x_1_L = self.xjLs[self.N_layers-1][0]
        return [(x_1_L-y)*self.derivative_of_activation_func(x_1_L)]
    
    def calculate_deltas(self,layer,deltas_layer):
        #when layer is three gives deltas for two (already made deltaL), when layer is 2 it gives deltas for 1 --> stop at layer 2
        if layer == 2:
            return [deltas_layer]
        layer_m1 = layer-1
        d_layer = [1] if layer == self.N_layers else range(1,self.layers[layer_m1]) 
        d_layer_m1 = range(1,self.layers[layer_m1-1]) 
        
        deltas_layer_m1_i = []
        
        for i in d_layer_m1:
            sum_of_j_paths = 0
            for j in d_layer:
                deltaj = deltas_layer[j-1]
                sum_of_j_paths += deltaj*self.weights[layer_m1-1][j-1][i] 
                self.count += 1
                #j-1 because the weights for j=1 are actually the first weights as j=0 is 1
            
            delta_layer_m1_i = self.derivative_of_activation_func(self.sjLs[layer_m1-2][i-1]) * sum_of_j_paths #-2 becuase no sjs for layer 1
            deltas_layer_m1_i.append(delta_layer_m1_i)
            
        return self.calculate_deltas(layer-1,deltas_layer_m1_i) + [deltas_layer]
    
    

In [62]:
class Plot:
    def simple_plot(self,X,preds):
        data = np.concatenate([np.array(X),np.array(preds).reshape(-1,1)],axis=1)
        df = pd.DataFrame(data, columns = ["x1","x2","y"])
        sns.scatterplot(data=df, x="x1", y="x2", hue="y")
        plt.xlim([-1, 1])
        plt.ylim([-1, 1])
        plt.show()
        
    def plot_ein(self):
        preds = self.calculate_preds(self.Xorig)
        if self.activation == "tahn": preds = np.sign(preds)
        self.simple_plot(self.Xorig, preds)
     
    def plot_eout(self):
        X = np.random.uniform(-1.0,1.0,(1000,2))
        preds = self.calculate_preds(X)
        if self.activation == "tahn": preds = np.sign(preds)
        y = self.find_actual_y(X)
        self.simple_plot(X, preds)
        self.simple_plot(X, y)
        preds = self.calculate_preds(X)
        plt.plot(preds,preds,"go")
        plt.plot(preds,y,"bo")
        plt.vlines(preds,preds,y, colors='k', linestyles='solid', label='')
        plt.show()

In [63]:
class Line():
    def create_line(self):
        if self.type == "linear regressor":
            self.mbs = np.random.uniform(10.0,0.0,self.input+1)
        if self.type == "PLA":
            self.mbs = np.random.uniform(1.0,0.0,self.input-1+1)
        if self.type == "circle":
            self.r = np.random.uniform(.8,0.0,1)
    def find_actual_y(self,X):
        if self.type == "PLA":
            return np.sign(np.matmul(self.add_thresholdCol(X)[:,:-1],self.mbs.T) - X[:,-1])
        if self.type == "linear regressor":
            return np.matmul(self.add_thresholdCol(X),self.mbs.T)
        if self.type == "circle":
            return [1 if x else -1 for x in np.sum(X**2,axis=1)>self.r**2]


    
    

In [64]:
class SGDGradient:
    
    def sgd_gradient(self):
        
        for layer in range(2,self.N_layers+1):
            layer_m1 = layer-1 #wij_2 where 2=layer is the weights between layer 1 (layer-1) and layer 2 (layer)
            delta_layer = self.deltas[layer-2] #delta layer 2 is the first delta at index 0
            dj_l = self.layers[layer-1]
            if layer==self.N_layers: dj_l +=1
            for j in range(1,dj_l):
                deltaj_l = delta_layer[j-1]
                for i in range(self.layers[layer_m1-1]):
                    wij_l = self.weights[layer_m1-1][j-1][i]#weights for layer 1 to go to layer 2 is at index 0
                    xi_layer_m1 = self.xjLs[layer_m1-1][i]
                    gradient = -self.n*xi_layer_m1*deltaj_l
                    self.count += 1
                    self.weights[layer_m1-1][j-1][i] = wij_l + gradient
                
        
        
        
    def sgd_epoch(self):
        vals = list(range(self.N))
        random.shuffle(vals)
        for v in vals[:1]:
            self.count = 0
            x, y = self.X[v],self.y[v]
            self.xjLs,self.sjLs = self.calculate_xjLs(x)
            self.deltaL = self.calculate_deltaL(x,y)
            self.deltas = self.calculate_deltas(self.N_layers,self.deltaL)
            self.sgd_gradient()
            print(self.count)
        return self.weights 

In [65]:
class ActivationFunc:
    def activation_func(self,sj):
        if self.activation == "relu":
            return max(0,sj)
        if self.activation == "tahn":
            return (e**sj-e**-sj)/(e**sj+e**-sj)

    
    def derivative_of_activation_func(self,s):
        if self.activation == "relu":
            return 0 if s<0 else 1
        if self.activation == "tahn":
            return (1-(self.activation_func(s))**2)
       

In [68]:


class NeuralNetwork(Line,Plot,ActivationFunc,Calculate_xjLs,CalculateDeltas,SGDGradient):
    def __init__(self):
        self.N = 100
        self.input = 5
        self.layers = [self.input+1,4,1]
        self.n = .1
        self.N_layers = len(self.layers)
        self.type = "linear regressor"
        self.activation = "relu"
        self.init_weights()
        self.createData()
        
        self.count = 0
        
        self.ein = self.mse(self.X,self.y)
        print("Pre Adjusting Ein %s" % self.ein)
        self.runSGD()
        
        print("Final Weights %s" % self.weights)      
        print("Num Iterations %s" %self.num_iter)
        self.ein = self.mse(self.X,self.y)
        print("Ein %s" % self.ein)
        
        self.eout = self.eout()
        print("Eout %s" % self.eout)
        
        #self.plot_ein()
        #self.plot_eout()
        
    
      
        
####################################################
    def runSGD(self):
        self.num_iter = 0
        hump = 0
        for x in range(1):
            wt = self.weights_list()
            self.num_iter += 1
            self.sgd_epoch()
            wt1 = self.weights_list()
            if np.sqrt(sum((wt-wt1)**2)) <.01: #np.linalg.norm
                hump += 1
                if hump>1000:
                    break
    def add_thresholdCol(self,X):
        return np.concatenate([[[1]for x in range(len(X))],X],axis=1)
    
    def createData(self):
        self.create_line()
        self.Xorig = np.random.uniform(-1.0,1.0,(self.N,self.input))
        self.y = self.find_actual_y(self.Xorig)
        self.X = self.add_thresholdCol(self.Xorig)

        
        
    def weights_list(self):
        change = []
        for layer in range(2,self.N_layers+1):
                layer_m1 = layer -1
                jss = [0] if layer == self.N_layers else range(self.layers[layer-1]-1)
                iss = range(self.layers[layer_m1-1]) 
                change += [self.weights[layer_m1-1][j][i] for i in iss for j in jss]
        return np.array(change) 
    
    def init_weights(self):
        self.weights = []
        for layer in range(1,self.N_layers):
            N_l = self.layers[layer]-1 if layer+1 != self.N_layers else self.layers[layer]
            N_lm1 = self.layers[layer-1]
            self.weights.append(np.random.uniform(-1.0,1.0,(N_l,N_lm1)))
        
    def mse(self,X,y):
        w = self.weights
        return sum([(self.calculate_xjLs(X[v])[0][-1][0] - y[v])**2 for v in range(len(X))])/len(X)
    
    def calculate_preds(self,X):
        X = self.add_thresholdCol(X)
        return [self.calculate_xjLs(x)[0][-1][0] for x in X]
        
    
    def eout(self):
        Xorig = np.random.uniform(-1.0,1.0,(100,self.input))
        X = self.add_thresholdCol(Xorig)
        y = self.find_actual_y(Xorig)
        return self.mse(X,y)

    

   

In [69]:
NeuralNetwork()

Pre Adjusting Ein 134.11922597883415
47
Final Weights [array([[ 0.14811362, -1.06543579, -0.88463932, -1.09803737,  0.45332922,
        -0.77225908],
       [-1.40406797,  0.17980799, -0.35437695,  0.58452447, -1.56258608,
         0.45126077],
       [ 0.56830913,  0.48879526,  0.87498181,  0.40012446, -0.72059293,
        -0.37618967]]), array([[ 0.13881588,  0.94110506, -0.62311481,  0.99119079]])]
Num Iterations 1
Ein 120.05073376769775
Eout 94.72058367678396
